# Correlating NYT Data with Market Data

### Loading in Data

In [2]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
import glob
import string
from dateutil.relativedelta import relativedelta
import numpy as np
import yfinance as yf
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [6]:
spy = yf.download("SPY")
demsent = pd.read_csv("./Data/DailyDemFiltered.csv")
repsent = pd.read_csv("./Data/DailyRepFiltered.csv")



xlk = yf.download("XLK")
xlv = yf.download("XLV")
xlf = yf.download("XLF")
xly = yf.download("XLY")
xli = yf.download("XLI")
xlp = yf.download("XLP")
xlu = yf.download("XLU")
xle = yf.download("XLE")
xlc = yf.download("XLC")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
listOfFunds = [spy , xlk , xlv, xlf, xly , xli, xlp, xlu, xle, xlc]

for etf in listOfFunds:
    etf["Log Returns"] = np.log(etf["Close"]/etf["Close"].shift(1))
spy.head()

,Open,High,Low,Close,Adj Close,Volume,Log Returns
Date,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.079659,1003200,NaN
1993-02-01,43.96875,44.25000,43.96875,44.25000,26.265144,480500,0.007087
1993-02-02,44.21875,44.37500,44.12500,44.34375,26.320782,201300,0.002116
1993-02-03,44.40625,44.84375,44.37500,44.81250,26.599014,529400,0.010515
1993-02-04,44.96875,45.09375,44.46875,45.00000,26.710312,531500,0.004175


In [ ]:
spy , xlk , xlv, xlf, xly , xli, xlp, xlu, xle, xlc = spy['2020-10-01':'2020-11-29'] , xlk['2020-10-01':'2020-11-29'] , xlv['2020-10-01':'2020-11-29'], xlf['2020-10-01':'2020-11-29'], xly['2020-10-01':'2020-11-29'] , xli['2020-10-01':'2020-11-29'], xlp, xlu, xle, xlc

In [6]:
def cov_cor(sentiment, logReturns):
    '''Returns a tuple containing the correlation and
    covariance respectively between two timeseries dataframes'''
    cor = sentiment.corr(logReturns)
    cov = sentiment.cov(logReturns) 
    return (cor, cov)

In [1]:
# np.cov(demsent, spy['2020-10-01':'2020-11-30']["Log Returns"])
# spy['2020-10-01':'2020-11-30']
# demsent.set_index("date")
print(demsent.info(), spy['2020-10-01':'2020-11-30'].info())

NameError: name 'demsent' is not defined